In [25]:
from requests import get
import pandas as pd
from datetime import datetime
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from modules.data_fetcher import download_historical_data

# Fear and Greed fetching

In [26]:

response_fng = get('https://api.alternative.me/fng/?limit=0&format=json').text

fng_df = pd.DataFrame(json.loads(response_fng)['data'])

fng_df.drop(columns=['time_until_update'],inplace=True)
fng_df.rename(columns={'timestamp':'date'},inplace=True)

fng_df['date'] = fng_df['date'].apply(lambda x: datetime.fromtimestamp(int(x)))
fng_df['value'] = fng_df['value'].astype(float)
fng_df.sort_values('date',inplace=True)
fng_df.set_index('date',inplace=True)
print(fng_df.shape)
fng_df.head()

(1849, 2)


,value,value_classification
date,,
2018-02-01 01:00:00,30.0,Fear
2018-02-02 01:00:00,15.0,Extreme Fear
2018-02-03 01:00:00,40.0,Fear
2018-02-04 01:00:00,24.0,Extreme Fear
2018-02-05 01:00:00,11.0,Extreme Fear


# History fetching

In [27]:
btc_df = download_historical_data('BTC-USDT','1day').loc[fng_df.index[0].strftime("%Y-%m-%d"):]
print(btc_df.shape)
btc_df.head()

(1852, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2018-02-01 01:00:00,1.517443e+09,10233.000000,9289.870332,10449.888887,8753.804313,397.353723,3.757247e+06
2018-02-02 01:00:00,1.517530e+09,9289.870332,8849.999994,9390.283108,7863.304872,600.803420,5.168853e+06
2018-02-03 01:00:00,1.517616e+09,8859.999995,9199.999443,9498.999721,8201.999000,268.980095,2.428066e+06
2018-02-04 01:00:00,1.517702e+09,9107.736677,8117.000002,9409.999999,7800.000000,382.837194,3.277363e+06
2018-02-05 01:00:00,1.517789e+09,8210.697671,6950.001000,8379.999999,6470.000001,943.740074,6.945778e+06


# Merging Fear and Greed with Price

In [32]:
btc_fng_df = pd.merge_asof(btc_df,fng_df, left_index=True, right_index=True, direction='nearest').loc["2021-11-28":]
btc_fng_df.head()

,Timestamp,Open,Close,High,Low,Amount,Volume,value,value_classification
Date,,,,,,,,,
2021-11-28 01:00:00,1.638058e+09,54713.6,57275.5,57443.9,53235.6,6760.098761,3.711303e+08,27.0,Fear
2021-11-29 01:00:00,1.638144e+09,57275.5,57777.4,58882.3,56664.5,7082.709914,4.083367e+08,33.0,Fear
2021-11-30 01:00:00,1.638230e+09,57777.4,56928.1,59200.0,55877.1,8159.206766,4.683648e+08,40.0,Fear
2021-12-01 01:00:00,1.638317e+09,56937.8,57187.5,59038.7,56450.0,7271.293808,4.177976e+08,34.0,Fear
2021-12-02 01:00:00,1.638403e+09,57187.5,56484.6,57372.8,55752.5,5966.009512,3.376922e+08,32.0,Fear


In [33]:
btc_fng_df['valueEMA'] = btc_fng_df.value.ewm(30).mean()
btc_fng_df.dropna(inplace=True)
btc_fng_df.head()

,Timestamp,Open,Close,High,Low,Amount,Volume,value,value_classification,valueEMA
Date,,,,,,,,,,
2021-11-28 01:00:00,1.638058e+09,54713.6,57275.5,57443.9,53235.6,6760.098761,3.711303e+08,27.0,Fear,27.000000
2021-11-29 01:00:00,1.638144e+09,57275.5,57777.4,58882.3,56664.5,7082.709914,4.083367e+08,33.0,Fear,30.049180
2021-11-30 01:00:00,1.638230e+09,57777.4,56928.1,59200.0,55877.1,8159.206766,4.683648e+08,40.0,Fear,33.475457
2021-12-01 01:00:00,1.638317e+09,56937.8,57187.5,59038.7,56450.0,7271.293808,4.177976e+08,34.0,Fear,33.613111
2021-12-02 01:00:00,1.638403e+09,57187.5,56484.6,57372.8,55752.5,5966.009512,3.376922e+08,32.0,Fear,33.268992


In [34]:
fig = make_subplots(
    rows=2, cols=1, subplot_titles=("Historical price", "Fear and Greed indicator")
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=btc_fng_df.index,
        open=btc_fng_df["Open"],
        high=btc_fng_df["High"],
        low=btc_fng_df["Low"],
        close=btc_fng_df["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="Historical Fear and Greed",
        x=btc_fng_df.index,
        y=btc_fng_df["valueEMA"],
    ),
    row=2,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Comparing price and fear and greed",
)
fig.show()

In [35]:
import plotly.express as px

z = btc_fng_df.drop(columns=['Timestamp']).corr(numeric_only=True)

fig = px.imshow(z, text_auto=True, aspect="auto")
fig.show()